In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
import sys
import os

In [2]:
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [3]:
warehouse_location = "hdfs://m1.local.br:9000/user/hive/warehouse"
hive_metastore_uri = "thrift://m1.local.br:9083"
defaultFS = "hdfs://m1.local.br:9000"

In [4]:
spark = SparkSession.builder \
    .appName("Search Movie - MovieLens") \
    .config("spark.sql.warehouse.dir", warehouse_location) \
    .config("spark.hadoop.fs.defaultFS", defaultFS) \
    .config("hive.metastore.uris", hive_metastore_uri) \
    .enableHiveSupport() \
    .getOrCreate()

spark

In [5]:
spark.catalog.refreshByPath(warehouse_location)

In [6]:
print("URI do Metastore do Hive:", spark.conf.get("hive.metastore.uris"))
print("Endereço do Hadoop (HDFS):", spark.conf.get("spark.hadoop.fs.defaultFS"))

URI do Metastore do Hive: thrift://m1.local.br:9083
Endereço do Hadoop (HDFS): hdfs://m1.local.br:9000


In [7]:
# Se conectando ao banco default
spark.sql("USE default")

DataFrame[]

In [19]:
def search_movie_by_title(title: str) -> list:
    res = spark.sql(f"SELECT * FROM movies WHERE upper(title) like '%{title.upper()}%' ORDER BY year DESC, title")
    
    res.show(20, truncate=False)

    return res.select('movieid').collect()

In [136]:
search_movie_by_title("Pursuit of Happyness")

+-------+-------------------------------+----+-------+
|movieid|title                          |year|genres |
+-------+-------------------------------+----+-------+
|47099  |The Pursuit of Happyness (2006)|2006|[Drama]|
+-------+-------------------------------+----+-------+



[Row(movieid=47099)]

In [137]:
spark.stop()